<span type="title">持有对象</span> | <span type="update">2018-07-24</span> - Version <span type="version">1.2</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解Java中的容器类以及其使用方法。在第一部分，快速浏览了这些容器类的类属关系，展示了如何利用泛型和接口来实现类型安全易用的容器类，展示了容器类的打印。在第二部分，介绍了常用的 List接口、Iterator接口、LinkedList类、Set接口、使用 LinkedList实现的 Queue 和 Stack 数据结构。之后还提到了 Map 这个稍微特殊的容器。</p><p class="card-text">在第三部分，主要讲解了使用容器类的几种方法：创建实例或者使用继承 or 实现，以实现 Iterator 和 继承 AbstractMap (实现享元) 为例进行了介绍。在这部分的最后提到了迭代器和数组的关系、Java内部的可选方法等细节问题。</p><p class="card-text">在第四部分，添加和补充了带有泛型的容器高级内容。首先探讨了几种不同的 Set 类型容器，特别提到了 HashSet、SortSet 用于存放自建类的注意事项。探讨了 Map 容器的几个实现：SortedMap 的使用方法、LinkedHashMap 以及LRU算法、WeakHashMap 以及Reference类型工作的原理。探讨了如何重载 hashCode 和 equals 来使用容器存放自有类。最后提到了 Collections 类中实用的静态方法：用于随机、线程安全、类型检查等实用的静态方法。</p></span>

# Java 容器概览

![](cp6.png)

如上图所示，其中虚线表示的是接口，接口体系中最重要的有 `Iterator, Collection, List, Set, Queue, Map` 和 `ListIterator` 。然后是代表不同特性的类，常用的有 `LinkedList, ArrayList, HashSet, HashMap` 等。

关于容器，它们支持了Java对于对象按照某种结构进行存放和取回，这在很大程度上扩展了对象的使用——因为程序往往不能在最开始知道自己要处理多少对象，因此不能逐个进行实例化，然后在程序启动的时候由启动器开辟恰好相同大小的内存空间。因此，对于数组Array而言，固定的尺寸往往很受限制，因此在用来解决问题的时候往往非常受限。对于面向对象而言，更是如此，通过容器类（集合类），Java可以灵活的通过某种结构保留对象，以便在合适的时候使用它们，这扩展了程序对于现实问题的建模和解决思路。

## 容器类快速上手

下面代码使用了一个和 Array 很像的 ArrayList 类，这个类的优点是，其不用限制每个元素的类型，以及其长度。但是，这样可能造成混乱，因此在得到元素的时候，比如通过直接类型声明来访问这个类型的方法，而通过泛型标签，我们可以限定AL类只能存留一种元素类型，这样就可以很方便的像Array一样使用AL了。下列代码展示了使用AL添加元素add()、删除元素remove()、根据下标获取元素get(index)的方法。

```java
//展示使用ArrayList包括其泛型持有对象和取出对象的方法
import java.util.*;
import static com.mazhangjing.Print.*;
public class ArrayListDemo {
    public static void main(String[] args){
        //-------------------构造、添加元素、获取索引元素-----------------
        Random rand = new Random();
        //使用ArrayList类型的列表容器，不用限定每个元素类型
        ArrayList orages = new ArrayList();
        for (int i = 0; i < 9; i++){
            //使用add方法添加新元素，不定长
            orages.add(new Orange(rand.nextInt(100)));
        }
        //可以添加别的类型，但是取出时需要注意
        orages.add(new Banana());
        for (int i = 0; i < 9; i++){
            //使用get(index)取出，但是需要注意，不像动态类型可以直接调用类型的方法，
            //编译器需要通过编译，则需要转型。
            print(((Orange)orages.get(i)).getName());
        }
        print(orages.get(9).getClass());
                                           
        //---------------------------使用泛型指定类型-------------------------
                                           
        //设置Orange类型的ALrrayist
        ArrayList<Orange> oranges = new ArrayList<Orange>();
        for (int i = 0; i < 9; i++){
            oranges.add(new Orange(3));
        }
        //自动向上转型
        oranges.add(new SmallOrange());
        //oranges.add(new Banana()); 错误
        for (int i = 0; i < 10; i++){
            //在有了泛型之后，就不用转型才能访问方法了
            oranges.get(i).getName();
        }
        //当使用了泛型后，可以用foreach遍历元素
        for (Orange e : oranges){
            print(e.getName());
        }
                                           
        //-----------------------使用容器接口提高扩展性-----------------------------
                                           
        //发生了向上转型，这里Coll是一个接口，使用接口而不是类作为类型的优点是：
        //在之后的代码中，调用的是接口的定义，避免了和类的耦合
        Collection oranges = new ArrayList<Orange>();
        oranges.add(new Orange());
        //因为Collection只是一个接口，必须向下转型到Arraylist才能使用这些方法
        print(((ArrayList) oranges).get(0).getClass());
                                           
        for (Object e: oranges){
            //虽然说是每个Collection均可使用foreach
            //但是因为不确定其中的类型，因此这里必须使用Object
            print(e.getClass());
        }
                                           
        //--------------将多个元素一次添加到容器的两种方法----------------------------
        int[] a = {1,2,3,234,12,12};
        //Collection是接口，Collections是一个静态工具类
        Collection<Integer> c = new ArrayList<Integer>();
        //addAll接受一些元素，添加到一个List
        Collections.addAll(c,1,213,123,123,213);
        //c.addAll 接受一个位置和一个List，注意，这里使用Arrays.asList生成列表（这个列表是固定长度的）
        //这里的Arrays也是一个静态工具类，含有很多重要方法，比如asList可以在Array和List类型之间转换
        ((ArrayList<Integer>) c).addAll(0,Arrays.asList(6,9,10));
        //不能在这里直接放一个未初始化的列表，常用c.addAll(Array.asList(e,e,e))来一次性将元素添加到List
        //或者使用Collections.addAll(c,e,e,e)，或者使用 c.addAll(List.of(e,e,e))。
        //((ArrayList<Integer>) c).addAll(0,{1,23,12});
        print(c); 
    }
}
class Orange {
    Orange() {}
    Orange(int n){num = n;}
    private int num = 0;
    int getName() {return num;}
}
class SmallOrange extends Orange {
    int getName() {return 2;}
}
class Banana {
    void getName() {}
}
```

注意在代码中是如何构造、添加一个、多个元素到容器，如何使用泛型指定类型，使用接口提高程序扩展性的。

## 容器的打印

注意，这里使用了 Map 容器，Map 容器在一个槽中保留一对对象，就像Python中的字典。使用 `Map<String:String>` 声明这两个对象的类型。使用 `put` 方法放置元素，而 Collection 则使用 `add` 放置元素。

```java
import java.util.*; import static com.mazhangjing.Print.*;
public class PrintDemo {
    public static Collection fill(Collection<String> c){
        c.add("cat");
        c.add("dog");
        c.add("bird");
        return c;
    }
    //因为Map需要传递两组参数进去，所以泛型声明使用 <String,String>
    public static Map fill(Map<String,String> map){
        map.put("name","CorkineMA");
        map.put("age","32");
        map.put("address","CCNU, Hubei Province");
        return map;
    }
    public static void main(String[] args){
        print(fill(new ArrayList<String>()));
        //print(fill(new List<String>())); List 是一个接口，不能被实例化
        print(fill(new LinkedList<String>()));
        print(fill(new HashSet<String>()));
        print(fill(new TreeSet<String>()));
        print(fill(new LinkedHashSet<String>()));
        print(fill(new HashMap<String,String>()));
        print(fill(new TreeMap<String,String>()));
        print(fill(new LinkedHashMap<String,String>()));
    }
}
class Exercise {
    public static class TestEx {
        public static void main(String[] args){
        //其实所有容器API分为两类，其一为每个槽对应一个元素的Collection接口扩展的类
        //另一个是每个槽对应两个元素的Map元素扩展的类
            Favorate f = new Favorate();
            ArrayList<String> al = new ArrayList<>();
            LinkedList<String> ll = new LinkedList<>();
            HashSet<String> hs = new HashSet<>(); //Set和List区别在于，前者元素不会重复
            LinkedHashSet<String> lh = new LinkedHashSet<>();
            TreeSet<String> ts = new TreeSet<>();
            for (int i = 0; i < 10; i++){
                al.add(f.next());
                ll.add(f.next());
                hs.add(f.next());
                lh.add(f.next());
                ts.add(f.next());
            }
            print(al); print(ll); print(hs); print(lh); print(ts);
        }
    }
}
class Favorate {
    private Random rand = new Random();
    String next(){
        switch (rand.nextInt(3)) {
            case 0: return "Star Wars";
            case 1: return "Game of thrones";
            default:
            case 2: return "Tom and Cat";
        }
    }
}
[Star Wars, Game of thrones, Tom and Cat]
```

# List

下面这个例子展示了 List 容器的大多数 API。包括判断包含`contains(E),containsAll(C)`、根据索引或者对象查找`indexOf,get`、根据上下索引截取`subList(i,ii)`、添加到指定位置`add(i,E)`、添加子容器`addAll(C)`、删除`remove(E)`、容器去重`retainAll(C)`。这些和Python所提供的List很相似，通过Java API手册即可了解。此外，`Collection.sort()` 可以就地排序， `.shuffle()` 可以随机化，这些在Python中需要导入包或者直接的函数，通常在Collection静态方法中存在。

```java
import static com.mazhangjing.Print.*; import typeinfo.pets.*; import java.util.*;
public class ListMe {
    public static void main(String[] args){
        Random rand = new Random();
        List<Pet> pets = Pets.arrayList(7);
        print("1" + pets);
        Hamster h = new Hamster();
        pets.add(h);
        print("2" + pets);
        //contains判断包含与否
        print("Is list contiains Hamster? " + pets.contains(h));
        print("3" + pets);
        pets.remove(h);//remove删除元素本身，返回布尔值
        Pet p = pets.get(2);//使用indexOf取出索引，使用get根据索引获取元素
        print("The index of" + pets.indexOf(p) +" is "+p);
        print("Remove index of 2: "+pets.remove(2));//remove也可以删除元素索引，返回
        //元素本身
        print("4" + pets);//add添加元素，可以指定位置，不返回结果，如果不指定位置返回布尔值
        pets.add(1,new Mouse());
        print("5" + pets);
        //使用sublist获取子元素，返回列表，参数为开始和结束的值，前包后不包
        List pp = pets.subList(0,3);
        print("The sublist is "+pp);
        //使用constainsAll判断一个列表是否包含另一个列表
        print("Father contains all son?"+pets.containsAll(pp));
        Collections.sort(pp);//就地排序，其中Collections包含了很多好用的函数式方法，包括乱序，排序
        print("The order sublist is "+pp);
        Collections.shuffle(pp);//可以重载为使用种子的排序
        print("The rand sublist is" + pp);
        print();
        //使用ArrayList本身的构造器复制一份列表
        List<Pet> copy = new ArrayList<Pet>(pets);
        //Arrays.asList可以将某种特定类型元素组合成一个固定长度的列表
        //是Array和Collection接口转换，此外还有Collection.toArray()
        List sub = Arrays.asList(pets.get(1),pets.get(3));
        print("The copy of pets list is " + copy);
        print("The sublist use Array.asList is "+sub);
        print("Now I will retain the copy from sublist" + copy.retainAll(sub));
        print("Now the copy retained is " + copy);
        copy.addAll(1,sub);//添加一个列表会重复往里面塞数据
        print("I add sub to copy, now the copy is "+copy);
        copy.removeAll(sub);//删除就不一样了，只要元素存在，就马上删除，不论几个
        print("I just add the sub to copy, and delete it later... \n now" +
                "The copy is empty? "+copy.isEmpty());
        pets.clear();
        pets.addAll(Pets.arrayList(4));//重新创建列表，添加到pets中
        print("The pets new is "+pets);//下列代码展示了array和list的转换
        Object[] o = pets.toArray();
        print("I just turn the list to array, \nIt need the type is Object[]"+o);
        print("You can get o use o[2] in array" + o[2]);

        Collection<My> mlist = new LinkedList<>();
        ((LinkedList<My>) mlist).add(new My());
        print(mlist);
    }

}
class My {
    public String toString() {
        return "My class!!!";
    }
}
1[Rat, Manx, Cymric, Mutt, Pug, Cymric, Pug]
2[Rat, Manx, Cymric, Mutt, Pug, Cymric, Pug, Hamster]
Is list contiains Hamster? true
3[Rat, Manx, Cymric, Mutt, Pug, Cymric, Pug, Hamster]
The index of2 is Cymric
Remove index of 2: Cymric
4[Rat, Manx, Mutt, Pug, Cymric, Pug]
5[Rat, Mouse, Manx, Mutt, Pug, Cymric, Pug]
The sublist is [Rat, Mouse, Manx]
Father contains all son?true
The order sublist is [Manx, Mouse, Rat]
The rand sublist is[Manx, Rat, Mouse]

The copy of pets list is [Manx, Rat, Mouse, Mutt, Pug, Cymric, Pug]
The sublist use Array.asList is [Rat, Mutt]
Now I will retain the copy from sublisttrue
Now the copy retained is [Rat, Mutt]
I add sub to copy, now the copy is [Rat, Rat, Mutt, Mutt]
I just add the sub to copy, and delete it later... 
 nowThe copy is empty? true
The pets new is [Manx, Cymric, Rat, EgyptianMau]
I just turn the list to array, 
It need the type is Object[][Ljava.lang.Object;@36f6e879
You can get o use o[2] in arrayRat
```

# Iterator

对于C++，所有的容器可以通过 iterator API接口调用而不用在意容器类型和容器特定的API接口。对于Java，也可以这样做。迭代器的好处在于，其提供了一套最高层次的接口，所有的容器类都遵循这套接口，所以，迭代器的方法，所有容器都通用。当然， 对于Python这种弱类型，用处不很大。

迭代器的接口有 `hasNext() 判断是否有下一个元素, next() 返回下一个元素, remove() 可选的功能`。迭代器的另一种选择是 for-each 语句，可以看到它们实现了相似的功能。当迭代结束后，会报错。当需要重新使用迭代器，必须重新从容器引用。 所有容器的迭代器接口在 `.iterator()` 方法上，其会返回一个迭代器对象，使用结束后，需要会重新赋值。

在下列例子中展示了 Iterator 的一个继承的接口 `ListIterator` 的使用，这个接口很适合充当导航，可以向前向后移动。

```java
import java.util.*; import static com.mazhangjing.Print.*; import typeinfo.pets.*;
public class Iter {
    public static void main(String[] args){
        List<Pet> pets = Pets.arrayList(12);
        Iterator<Pet> it = pets.iterator();//使用ins.iterator()返回迭代器对象
        //使用hasNext和next可以遍历循环
        while (it.hasNext()){
            Pet p = it.next();
            print("" + p + ", " + p.id());
        }
        //替代迭代器的另一种选择是foreach
        for (Pet p : pets){
            print(p + " " + p.id());
        }
        //当迭代器迭代完毕后，就没有元素了，next返回NoSuchElement错误
        it = pets.iterator(); //重新引用
        for (int i = 0; i < 5; i++){
            it.next();
            it.remove();//这个很有意思，它删除了list中的元素
        }
        print(pets);
    }
                   
    //--------------------------下列代码展示迭代器整合不同容器类的能力--------------------------
    public static class IterPro {
        public static void main(String[] args){
            //用来展示iter跨容器类型的适用性
            ArrayList<Pet> pets = Pets.arrayList(10);
            HashSet<Pet> hs = new HashSet<>(pets);
            LinkedList<Pet> ll = new LinkedList<>(pets);
            Me m = new Me();//只用将这些容器类型的迭代器传递给这个方法就行了
            //迭代器似乎是一种这些容器的接口？？
            m.display(hs.iterator());
            m.display(ll.iterator());
        }
    }
                   
    //---------------------一个继承自Iterator的子接口ListIterator，可以向前向后移动-------------------------
    public static class ListIter {
        public static void main(String[] args){
            //ListIterator只能作用于List，但功能更强大，可向前向后，调用next()指针后移，nextIndex()
            //如果在next()之后调用，指针会再移动一次!!!!!!
            List<Pet> pets = Pets.arrayList(7);
            ListIterator<Pet> i = pets.listIterator();
            Pet p2 = Pets.randomPet(); print("The random pet is "+p2);
            i.add(p2);//add会替换掉当前指针指向前的一个元素，调用previous()可查看
            print("I got " + i.previous() + " at " + (i.previousIndex()+1));
            //注意在这个例子中，调用过previous()后，指针指向了0之前，因此再调用previousIndex会返回-1
            while (i.hasNext()){
                //注意，调用next()后，再调用nextIndex()会返回下一个元素，注意下面这种情况
                print(i.nextIndex() + " " + i.next() + "," + i.nextIndex());
            }
            Pet p = Pets.randomPet(); print("The random pet is "+p);
            i.set(p);//set会替换最后一个元素，因此可以看到，在下方返回的时候，并不是和上面
            //完全镜像的,只有在最后一个元素被call之后才可以set
            while (i.hasPrevious()){
                print(i.previousIndex() + " " + i.previous());
            }
        }
    }
}
class Me {
    void display(Iterator<Pet> i){
        while (i.hasNext()){
            print(i.next());
        }
    }
}
The random pet is Manx
I got Manx at 0
0 Manx,1
1 Rat,2
2 Manx,3
3 Cymric,4
4 Mutt,5
5 Pug,6
6 Cymric,7
7 Pug,8
The random pet is Cymric
7 Cymric
6 Cymric
5 Pug
4 Mutt
3 Cymric
2 Manx
1 Rat
0 Manx
```

对于 ListIterator，调用 `next()`指针后移，`nextIndex()` 返回下一个元素的指针，但是不移动。对于 LI，特别需要注意指针位置问题，以及添加或者删除元素`add, remove` 对于指针的影响！！！ 

# LinkedList

LinkedList 是一种继承自 List 接口的类，和 ArrayList 不同的是， LinkedList 擅长顺序读写，以及在一个 List 中插入和删除元素，在这种情况下速度较快。而 ArrayList则优于随机读写。（可能这里的LinkedList内部采用了和C类似的链表的技术）。

下面简要介绍了其操作：添加 `add() addAll() addF() addL()`、替换 `set(i,E)`、获取 `get() getF() getL()`、索引 `indexOf() lastIndexOf()`、删除 `remove() rmF() rmL()`。因为 LinkedList 多用于实现一些像是 Queue 和 Stack 等数据结构，因此其含有很多相似的方法，我个人建议对于普通 List 操作使用 addFirst(), getFirst(), removeFirst() 这种方法，而像是实现的那些高级数据结构，使用 peek() push() pop() poll() 这样不容易混淆。

这里顺便提一下， `pop()` 用来 Stack 的第一个元素弹出（类似removeFirst）， `push()` 用来 Stack 压入到第一个元素（类似于addFirst）， `poll()` 用来 Queue 的第一个元素取走（removeFirst）。 `peek()` 用来 Queue 查看第一个元素（getFirst）。在下面会详细介绍。

```java
import typeinfo.pets.*; import java.util.*; import static com.mazhangjing.Print.print;

public class LinkedListDemo {
    public static void main(String[] args){
        //添加 add(int,E) addFirst() add() = addLast() addAll()
        //替换 set(int,E)
        //获取 get(int) getFirst() = element() ≈ peek() = peekFirst() getLast() ≈ peekLast()
        //索引 indexOf(E) lastIndexOf(E)
        //删除 remove(int) removeFirst() = pop() ≈ poll() = pollFirst() removeLast() ≈ pollLast() remove(Obj)
        //入栈 push(E) 出栈 pop()
        LinkedList<Pet> pets = new LinkedList<Pet>(Pets.arrayList(5));
        print(pets + "\n\nADD DEMO: \n\n");
        Pet p = Pets.randomPet(); print("Random pet now is "+p);
        print("在开头和结尾添加这只宠物，\n之后获取原本位于1,2位置的两个，组成list，作为列表添加在末尾");
        pets.add(p); pets.add(0,p); pets.addAll(Arrays.asList(pets.get(2),pets.get(3)));

        print(pets + "\n\nSET & GET DEMO: \n\n我特别喜欢这个选中的动物，前三个必须是它！");
        pets.set(0,p);pets.set(1,p);pets.set(2,p);
        print(pets);
        print("现在用五种方式获取第一个元素的值：peek/peekfirst/element/get0/getfirst:");
        print(""+pets.peek() + pets.element() + pets.peekFirst() + pets.get(0) + pets.getFirst());
        print("是不是出错了？看看最后一个值"+pets.getLast() + pets.peekLast());

        print("\nREMOVE&PUSH DEMO:\n");
        print("使用removeFirst/pollFrst/remove(o)删除前三个值：");
        pets.removeFirst(); pets.pollFirst(); pets.remove(p);
        print(pets);
    }
    public static class Exercise {
        public static void main(String[] args){
            LinkedList<Integer> n = new LinkedList<>();
            n.addFirst(0);
            ListIterator<Integer> i = n.listIterator();
            Random rand = new Random();
            for (int j = 0; j < 5; j++){
                i.next();
                i.add(rand.nextInt(100));
                i.previous();
            }
            n.removeFirst();
            print(n);
        }
    }
}
[Rat, Manx, Cymric, Mutt, Pug]

ADD DEMO: 
Random pet now is Cymric
在开头和结尾添加这只宠物，
之后获取原本位于1,2位置的两个，组成list，作为列表添加在末尾
[Cymric, Rat, Manx, Cymric, Mutt, Pug, Cymric, Manx, Cymric]

SET & GET DEMO: 
我特别喜欢这个选中的动物，前三个必须是它！
[Cymric, Cymric, Cymric, Cymric, Mutt, Pug, Cymric, Manx, Cymric]
现在用五种方式获取第一个元素的值：peek/peekfirst/element/get0/getfirst:
CymricCymricCymricCymricCymric
是不是出错了？看看最后一个值CymricCymric

REMOVE&PUSH DEMO:
使用removeFirst/pollFrst/remove(o)删除前三个值：
[Cymric, Mutt, Pug, Cymric, Manx, Cymric]

```

# Stack 

栈指的是后进先出的容器（LIFO），也就是最后进入的元素第一个出去，就像弹簧盒，最后压入栈的元素第一个弹出栈。使用 LinkedList 来实现 Stack 接口。因为某些历史原因，不要使用 Java 的 Stack 类。而仅仅使用一个较窄的接口即可：`Stack<E> x = new LinkedList<>();`

```java
import java.util.LinkedList; import java.util.Random;
import static com.mazhangjing.Print.print;

public class Stack<T> {
    private LinkedList<T> list = new LinkedList<>();
    public T pop(){return list.removeFirst();}
    public T peek(){return  list.getFirst();}
    public void push(T n){ list.addFirst(n);}
    public boolean isEmpty() {return list.isEmpty();}
                       
    public static void main(String[] args){
        Stack<Integer> s = new Stack<>();
        Random rand = new Random();
        s.push(rand.nextInt(100));
        print(s.peek());
        print(s.pop());
        print(s.isEmpty());
                                           
        //--------------------一个使用Stack的小例子---------------------
        Stack<Character> s = new Stack<>();
        boolean ready = false;
        for (char i : "+U+n+c---+e+r+t---+a-+i-+n+t+y---+-+r+u--+l+e+s--".toCharArray()){
            if (ready) {s.push(i);System.out.print(i);ready=!ready;}
            if (i == '-'){
                s.pop();
            } else if (i == '+'){
                ready = true;
            }
        }
        print(s.list);
    }
}
28
28
true
Uncertainty-rules[l]
```

**使用范型**

注意这里的一个语法细节，我们在类的定义中使用了范型，类名`<T>`告诉编译器，这是一个参数化类型，其中的类型参数在实际使用中需要被替换成实际的参数，比如Integer，String等。这意味着，我们定义了一个接受类型为 T 的 Stack 类。也就是说，这里面的大多数代码，是关于 T 类型进行的操作。

注意，这里的 `Stack` 是一个类型，而 `T` 也是一个类型。通过这种范型，我们将这两种类型的关系展示了出来，其暗含了这样的意味：Stack 是 T 的一个数据结构。

# Set

Set 是一种特殊的 List，它们没有继承关系，Set 和 List 都是由 Collection 接口继承而来的接口。 Set 表示元素不重复。其有三种实现，`HashSet` 速度快，无序，因为我们使用 Set 的原因常常是因为想要判断一个元素是否存在，因此 `HashSet` 较为常用。`TreeSet` 进行了自动排序，`LinkedHashSet` 继承自 `HashSet`, 在保证速度前提下，按照添加的顺序排序。

```java
import java.util.*; import static com.mazhangjing.Print.print;

public class SetDemo {
    public static void main(String[] args){
//        Set<Integer> s = new HashSet<Integer>();//无序，快速，常用
//        Set<Integer> s = new LinkedHashSet<>();//按照添加顺序，对于字符有时有用
        Set<Integer> s = new TreeSet<Integer>();//升序，耗费资源多
        Random rand = new Random();
        for (int i = 0; i < 100; i++){
            s.add(rand.nextInt(7));
        }
        print(s);
    }
}
[0, 1, 2, 3, 4, 5, 6] //注意，我们往这个Set中添加了100个元素！而现在只有7个。
```

# Map

Map 就像 Python 中的 dict 类型一样，或者说像是二维无序可变数组（C）。其在一个槽中保存了两个元素，在定义时需要说明，比如 `Map<String,Integer> x = new HashMap<>()`; 需要注意，Map 也是一个接口，其实现有 `HashMap, TreeMap, LinkedHashMap`。此外，Map可以嵌套，比如 `Map<String,List<Integer>>` 或者 `Map<String,Map<String,Map<String,Integer>>>`，虽然它看上去不如 Python 或者 JSON 那样灵活，对于每个槽可以保存不同的元素层级，但是，其实 `Map<Objcet,Object>` 可以在第二个 Object 中保存另一个 Map ，就像这样：

```java
Map<Object,Object> x = new HashMap<>();
Map<Object,Object> y = new HashMap<>();
x.put(27,12);
x.put("Str",23);
y.put(4,12);
x.put("Lev",y);
print(x); //{Str=23, 27=12, Lev={4=12}}
```
说回正题，Map 的使用主要是两个方法， `get(K), put(K,V)`。下面这个例子展示了使用 Map 来统计随机数在随机 100000 次之后的频次。

```java
import java.util.*; import static com.mazhangjing.Print.print;

//用来展示Map用法的示例代码 定义、get(K)、put(K,V)
public class Map_QueueDemo {
    public static void main(String[] args){
        Map<Integer,Integer> m = new HashMap<>();
        Random rand = new Random();
        Integer va;
        for (int i = 0; i < 100000; i++ ){
            int ne = rand.nextInt(20);
            va = m.get(ne);
            if (va == null) va = 0;
            m.put(ne,va+1);
        }
        print(m);
    }
}
{0=5027, 1=5078, 2=4917, 3=5023...}
```

# Queue

## Queue 的基本用法

Queue 队列是一种很常用的数据结构，其表示先进先出（FIFO）。Java只提供了 Queue 的接口，使用 LinkedList 实现这个接口来使用 Queue。

Queue 的方法中 `peek(), element()` 获取头部元素，其中后者获取不到会报错；`poll()，remove()` 删除头部元素，对于后者，没有元素可供删除会报错。`add(E), offer(E)` 常用 `offer` 来添加元素到尾部，后者不进行检查，速度更快。

下面的例子展示了如何使用 Queue 这种结构，我们新建了命令类，以及用来接收命令输入，使用 Queue 接口保存到 LinkedList 中，然后按照 Queue 的先进后出的模式取出命令并执行。

```java
public class QueueDemo {
    public static void main(String[] args){
        //Queue只实现了接口，因此需要使用LinkedList向上转型，在声明的时候
        //必须声明q = new LinkedList() 否则会报错，不能使用 q.offer()
        Queue<Integer> q = new LinkedList<>();
    }
    public static class QueueEx {
        public static void main(String[] args){
            OS os = new OS();
            os.setCMD(new Command("cd /"));
            Queue<Command> q = os.setCMD(new Command("rm -rf *"));
            os.runCMD(q);
        }
    }
}
class Command {//命令类
    Command(String s){cmd = s;}
    private String cmd = "Command";
    public String operation(){return cmd;}
}
class OS {//用来接受CMD输入，保存在Queue中，并且执行一组CMD语句的类
    private Queue<Command> q = new LinkedList<>();
    Queue<Command> setCMD(Command c){
        q.offer(c);
        return q;
    }
    void runCMD(Queue<Command> cs){
        while (cs.peek() != null){
            Command c = cs.poll();
            print(c.operation());
        }
    }
}
```

## PriorityQueue 优先级队列

队列可以有优先级，对于 VIP，可以插队，所以先进不一定后出，使用 PriorityQueue，指定一种顺序，然后在 `offer` 调用的时候，会自动插入新元素到合适的位置。

```java
Random rand = new Random();
//指定长度（可选）和排序方法（可选，对于非基本包装器，需要自己实现排序方法）
PriorityQueue<Integer> qi = new PriorityQueue<>(15,Comparator.naturalOrder());
for (int i = 0; i < 15; i++){//什么顺序？
    qi.offer(rand.nextInt(50)+10);
} 
print(qi);
[12, 13, 14, 30, 33, 30, 25, 53, 43, 58, 48, 57, 58...]
//感觉不是太有顺序啊
```

# 容器使用指南

## 使用 Iterator 接口 或 Abstract 继承

现在我们的问题是，如何去使用一个容器？我们看到有 Iteration, Collection interface，也有 AbstractCollection 这个纯实现了 Collection 的抽像基类，那么我们是要继承还是要实现，要实现的话，实现 Iteration 还是 Collection ？

因为继承会占用一个关键字，如果仅仅是需要一个类来进行遍历，那么推荐实现 Iteration。实现 Iteration 对于类的限制最少，如果这可以满足需求，那么就使用迭代器（或ListIteration）。否则，继承 Collection。

对于实现而言，下面的 `Trials` 就是一个很好的例子。

## 基于 Iterator 实现的扩展：适配器 or 内部类

第二个问题是，我们应该直接实现，还是应该添加一个中间件，让这个中间件实现后，将其作为内部类使用？这取决于我们面临的状况。对于 “迭代器设计模式” 而言，我们的抽像接口是 “包装类中的iterator方法” 和 “迭代器类中的next()和hasNext()” 方法。因此，可以最好是将多个迭代器去实现不同功能的迭代器接口，比如顺序、逆序或者随机顺序。而包装类的 iterator 方法则调用这些迭代器实现。

为了方便修改（因为当修改了包装类，那么迭代器的实现中next()和hasNext()的方法也必须修改，它们都是实现），将多个迭代器实现的类包括在包装类中作为内部类，如下的 SuperTrials 类所示。

但是，当我们不能修改一个已经实现的类，或者仅仅希望添加代码，而不是抛弃这个类 `Trials` 的话，使用适配器模式为这个 `Trials` 类写一个能够实现反向迭代的迭代器的适配器。通过 `reverse()` 返回这个迭代器，而由于继承，使用 `iterator()` 可以返回正常的迭代器。当然，这种方法更容易出问题，并且难以继续扩展和复用。

```java
import java.util.*;
import static com.mazhangjing.Print.print;

class Trial {
    Trial(int id){this.id = id;}
    private int id = 0;
    public String toString() {
        return ""+id;
    }
}
//-------------------------使用Iterator实现更方便-----------------------
class Trials implements Iterator {
    protected LinkedList<Trial> tlist = new LinkedList<>();
    protected int ptr = 0;
    Trials(List<Trial> ts){
        tlist.addAll(ts);
    }
    public Trial next() {
        return tlist.get(ptr++);
    }
    public boolean hasNext() {
        if (tlist.size() > ptr) return true;
        else return false;
    }
}
//------------------------多重继承实现多种迭代---------------------------
class SuperTrials {
    //相比较打补丁的方式，这种方式更优雅、清晰，但是也丢弃了之前写的Trials类。
    private LinkedList<Trial> tlist = new LinkedList<>();
    private class MIterator implements Iterator{
        private int ptr = 0;
        MIterator(List<Trial> ts){
            tlist.addAll(ts);
        }
        public Trial next() {
            return tlist.get(ptr++);
        }
        public boolean hasNext() {
            if (tlist.size() > ptr) return true;
            else return false;
        }
    }
    private class RIterator implements Iterator {
        private int ptr;
        RIterator(List<Trial> ts){
            tlist.addAll(ts);
            ptr = tlist.size();
        }
        public Trial next() {
            return tlist.get(ptr--);
        }
        public boolean hasNext() {
            if (ptr > 0) return true;
            else return false;
        }
    }
    public Iterator<Trial> iterator(List<Trial> ts, boolean reverse){
        if (!reverse) return new MIterator(ts);
        else return new RIterator(ts);}
}
//------------------------适配器实现反向迭代-----------------------------------
class FXTrials extends Trials {
    //这也是一种解决问题的思路，其使用了继承，然后添加了一个反向的方法，好像给Trials打补丁
    private int r_ptr;
    FXTrials(List<Trial> ts){
        super(ts);
        r_ptr = ptr;
    }
    public Iterator<Trial> reverse() {
        return new Iterator<Trial>() {
            public boolean hasNext() {
                if (r_ptr > 0) return true;
                else return false;
            }
            public Trial next() {
                return tlist.get(r_ptr--);
            }
        };
    }
}

Trials ts = new Trials(Arrays.asList(new Trial[]{new Trial(23),new Trial(13)}));
//Array.asList(new E[]{e1,e2..}) or List.of(e1,e2,...);都可以将元素快速添加到List
while (ts.hasNext()){
    print(ts.next());
}
```

## 基于 AbstractMap 实现的享元

我认为有两种接口，其一为抽象类的顶层类，和抽象类共同工作，这种接口是用于类库之间的交互和较大项目的实现使用，其一般含有的方法定义很多，对于使用而言，没有必要花那么大的力气。其二为一些专门的接口，比如Readable、Iterable接口，其生来就是被随处使用的，方法简单，容易实现。

使用容器，这些接口肯定是庞大的，因此，继承AbstractMap抽象类来实现我们自己的类比直接实现Map接口要省力许多。对于简单的类，我们可以直接实现Iterator接口，但是对于Mao这种，Map的接口需要实现的方法过于复杂，因此考虑对其一个子类使用继承。实际上，所有的类库中的容器都是由这些 AbstractXXX 继承而来，而不是重新实现了一遍接口。

**享元**指的是这样一种机制，我们不在一个新类中去创建一个现有类的实例，这样对性能有影响，并且充斥着大量内部包裹的，不可对外见的无用的这些内部类的代码和浪费，比如我们在一个自有容器中包裹一个ArrayList，就造成了大量的代码浪费。享元是试图在继承体系中去修改现有类的部分行为来为我所用，因为没有实例化任何对象，性能开销很小。但是需要对这些类有足够的了解。最好是查看Map的源代码。

下面的例子介绍了一个自造容器，其不可修改和删除元素，我们并未拦截这些操作，而是从根本上——Map用于删除和修改元素的方法上进行重载，返回了错误的声明来做到的。

首先介绍一下Map工作原理：当我们遍历一个Map时，Map内部通过 entrySet() 方法返回一个 EntrySet 对象，而这个EntrySet 包含一个叫做 iterator 的方法，这个方法将返回 EntrySet 的一个 Iterator 迭代器，这个迭代器使用了 Iterator 接口，因此可以调用 next 和 hasNext 方法，这个迭代器具有一个叫做 remove 的方法可以删除元素。每次调用将返回一个 Entry 对象，这个对象构造需要传入一个 int（HashCode？），指明其位置，其具有 getValue() setValue() 方法，用来进行元素的修改。

因此，我们唯一需要做的就是重载一个 entrySet 方法，而这个方法将提供一个 EntrySet 的对象。在下面代码中，可以看到我们构造的静态类 Entry 和 EntrySet，目的就是为了包装到这个方法中返回。至于只读，我们只需要对 EntrySet.Iterator 类的 remove 方法进行限制，对 Entry 的 setValue 方法进行限制即可。需要注意，对于 EntrySet ，如果不是非要 remove ，那么直接 new EntrySet，然后代理方法也可行，不过不纯粹。

```java
import java.util.*;
import static com.mazhangjing.Print.*；

public class iMap {
    public static final String[][] DATA = {
        {"China","BeiJing"}, {"American","W.D.C"}, {"U.K.","London"},
            {"Taiwan","Taipei"},{"HongKong","HongKong"}
    };
    private static class FlyweightMap extends AbstractMap<String,String> {
        private static class Entry implements Map.Entry<String,String> {
            int index;
            Entry(int index) {this.index = index; }
            public boolean equals(Object o) {
                return DATA[index][0].equals(o);
            }
            public String getKey() { return DATA[index][0]; }
            public String getValue() { return DATA[index][1]; }
            public String setValue(String value) { 
            //因为我们要打造一个不支持修改的列表，因此需要屏蔽这个Entry.setValue方法
                throw new UnsupportedOperationException();
            }
            public int hashCode() {
                return DATA[index][0].hashCode();
            }
        }
        /**接着要实现一个EntrySet，这个Set应该持有size信息，能够从所有元素中获取子串
         * 其次，还要实现一个迭代器，以返回Entry元素
         * 当然，这一步不是必须的，可以直接在entrySet中返回一个LinkedSet即可
         * 但是我们在这里需要禁止remove元素(Entryset的Iter迭代器的remove方法)，
         * 所以就实现了EntrySet*/
		static class EntrySet extends AbstractSet<Map.Entry<String,String>> {
			private int   size;
			EntrySet(int size) {
				if (size < 0) this.size = 0;
				else if (size > DATA.length) this.size = DATA.length;
				else this.size = size;
			}
			public int size() { return size; }
			//使用泛型的迭代器，注意，这里声明的是其基类Map.Entry，而不是Entry
            //对于拒绝删除元素的Map而言，要屏蔽remove方法
			private class Iter implements Iterator<Map.Entry<String,String>> {
				private Entry entry = new Entry(-1);
				public boolean hasNext() {
					return entry.index < size - 1;
				}
				public Map.Entry<String,String> next() {
					entry.index++;
					return entry;
				}
				public void remove() {
					throw new UnsupportedOperationException();
				}
			}
			//返回一个迭代器方法iterator，这个方法产生一个迭代器，用以迭代列表中的元素
			public Iterator<Map.Entry<String,String>> iterator() {
				return new Iter();
			}
		}
		//最后，终于能够返回一个EntrySet了，将此设置为static域，使用entrySet返回即可
		private static Set<Map.Entry<String,String>> entries = new EntrySet(DATA.length);
		public Set<Map.Entry<String,String>> entrySet() {
			return entries;
		}
    }
    /**这个方法被用来返回一个Map，这个Map为匿名内部类，重载了entrySet方法，返回了前size个item的信息*/
	static Map<String,String> select(final int size){
		return new FlyweightMap() {
			public Set<Map.Entry<String,String>> entrySet() {
				return new EntrySet(size);
			}
		};
	}
	static Map<String,String> map = new FlyweightMap(); //内部保存一个map
	//静态方法，用来截取一些不同长度map，或者直接返回这个map
	public static Map<String,String> capitals() {
		return map;
	}
	public static Map<String,String> capitals(int size) {
		return select(size);
	}
	static List<String> names = new ArrayList<>(map.keySet()); //从map中调用keySet方法返回key的set列表
	public static List<String> names() { return names; }//静态方法，返回名称列表
    //静态方法，返回截取的名称列表
	public static List<String> names(int size) {
		return new ArrayList<String>(select(size).keySet());
	}
	public static void main(String[] args){
		print(capitals(10));
		print(map.get(20));
		//map.put("China","WuHan"); //UnsupportedOperationException
        //map.remove("China"); //UnsupportedOperationException
	}
}
```

## 注意：Array 和 迭代器并无关系

```java
String[] s = {"Hello","Corkine"};
//s.hasNext() 错误，不能使用Iterator接口方法
List<String> ss = new LinkedList<>(Arrays.asList("Hello","World"));
ss.iterator().hasNext();
```

注意，Array 不能使用 Iterator 接口的方法！虽然它也可以使用 for-each。

## 注意：容器内部的可选操作

不一定需要将所有的接口方法全部实现，因为我们有时候并不需要全部接口方法。而重新设计接口的话，会造成无所谓的浪费和混淆。比如下面这个例子，对于一些依靠固定尺寸数据结构的方法，比如 asList，返回的是不可更改的列表，虽然我们转型成了 List，但是调用 add 等方法在运行时依旧会出错，这在语法上是没有问题的。这就是可选操作带来的副作用：UnsupportedOperation异常。

此外，对于一些尚未完全实现的方法，也可以先这么搞。

```java
//对于Collection类，添加和删除都是可选操作，实现类不必须提供定义。这是为了让一些情况，比如不可修改的Collections
//类似于只能实现部分方法的特例复用Collection提供的方法定义。

List<String> l = Arrays.asList("hello","from","corkine","ma");
List<String> l2 = new ArrayList<>(l);
//l.add("something"); //UnsupportedOperationException
l2.add("something");
```

# Java 容器进阶


## Set 接口容器

Set 接口相关的容器类有：HashSet, LinkedHashSet, 一个叫做 SortSet 的接口和其唯一的实现 TreeSet。

上文展示了Java内置的类作为元素的容器操作，当我们使用容器存放自己的类时，可能出现问题。对于 Hash系列的容器而言，必须实现：

- hashCode() 用于快速存储 
- equals() 用来判断是否重复 

对于 hash 和 LinkedHash 而言，这两个都是必须的。而对 sortSet 和 treeSet 而言，除此之外，还要实现 Comparable 接口的方法，用于排序。下面是一个示例：

```java
class TypeA {
    int keep;
    TypeA(int keep) { this.keep = keep;}
    public boolean equals(Object obj) {
        if (obj.getClass().getSimpleName() == "TypeA") {
            return ((TypeA)obj).keep == this.keep;
        } else return super.equals(obj);
    }
    public String toString() {
        return super.toString() + ": " + keep;
    }
    public int hashCode() {
        return super.hashCode();
    }
}
Random rand = new Random();
Set<TypeA> set = new HashSet<>();

//对于同一个引用，set只会添加一次
TypeA a = new TypeA(7);
for(int i = 0; i < 10; i++){
    set.add(a);
}
print(set);

TypeA b = new TypeA(7);
printf("b equals a? %b \nb == a? %b",b.equals(a), b == a); //false true
set.add(b);
print("\n"+set);
```

下面的例子将演示 sortset 接口的常用方法：截取子串 subSet headSet、获取头尾元素 first last higher。

```java
//TreeSet是唯一实现了SortSet的类，任何使用sortset的类必须除了实现hash和equals，还必须实现Comparator。
SortedSet<String> s = new TreeSet<>();
for (String x : "hello central hello china normal university".split(" ")){
    s.add(x);
} print(s); //没有发现重复的元素
//常见的SortSet方法：截取子串、获取头尾元素
printf("first ele: %s\nlase ele: %s\nhiger than china ele: %s\n"
        ,s.first(),s.last(),((TreeSet<String>) s).higher("china"));
printf("subset from [central to normal): %s\n",((TreeSet<String>) s).
                        subSet("central","normal"));
printf("head subset to normal: %s\n",((TreeSet<String>) s).headSet("normal"));

[central, china, hello, normal, university]
first ele: central
lase ele: university
higer than china ele: hello
subset from [central to normal): [central, china, hello]
head subset to normal: [central, china, hello]
```

## Map 接口容器

Map 包括：HashMap、weakHashMap、TreeMap、LinkedHashMap。

Map 对类型的需求和 List 类似，必须实现 HashCode 和 equals 方法，如果需要排序，则还要实现 Comparable 方法。

常见的Map实现的用途:

- HashMap 使用HashCode替代线性搜索，提高速度
- LinkedHashMap 在HashMap基础上使用链表保存插入次序，在迭代中速度更快
- WeakHashMap 映射之外没有指向某个键的引用，就回收它
- TreeMap 自动排序（红黑树）的MAP，需要实现Comparable方法

**SortedMap DEMO**

下面介绍了 SortedMap 和其唯一的实现：TreeMap 的使用方法，大体和上面的Set类似， tailMap 截取， keySet、entrySet 返回SET， firstKey、lowerKey 返回 KEY。

```java
SortedMap<Integer,String> s = new TreeMap<>();
Random rand = new Random(30);
for (String x : "Hello from Central China Normal University".split(" ")){
    s.put(rand.nextInt(10),x);
} printf(""+s); //排过序的
//下面展示了其方法，可以返回键set、键对set、首键、尾键、比较的键
printf("keyset: %s\nentry set: %s\nfirst key: %s\nlower key than 8: %s\n",
        s.keySet(),s.entrySet(),s.firstKey(),((TreeMap<Integer, String>) s).lowerKey(8));
//此外，还可以截取Map（因为排过序了）
printf("tail map from 5: %s\n",((TreeMap<Integer, String>) s).tailMap(5));

{4=China, 5=Central, 6=Normal, 8=University}
keyset: [4, 5, 6, 8]
entry set: [4=China, 5=Central, 6=Normal, 8=University]
first key: 4
lower key than 8: 6
tail map from 5: {5=Central, 6=Normal, 8=University}
```

**LinkedHashMap DEMO**

LinkedHashMap 和 HashMap 一样，对于所有的元素都实现了散列。但是，它依旧按照插入的链表顺序进行返回元素。当然，也可以不保存这个顺序，传入参数，可以按照LRU算法，将那些没有访问过的元素自动排序，放在最前面，下面的6和9访问多次，可以看到多次访问后，其顺序被拉在了后面。

```java
LinkedHashMap<Integer,String> s =new LinkedHashMap<>(16,0.75f,true);
Random rand = new Random(42);
StringBuilder sb;
char c; int m = 0;
for (int i = 0; i < 10; i++){
    sb = new StringBuilder();
    m = 0;
    while (true) {
        c = (char)rand.nextInt(100);
        if (Character.isAlphabetic(c)){
            m++;
            sb.append(c);
        }
        if (m == 5) {break;}
    }
    s.put(rand.nextInt(10),sb.toString());
} print(s);
for (int i = 0; i < 100; i++){
    s.get(6);
    s.get(9);
} print(s); //由于过多的访问6和9，因此其顺序靠后了。

{2=TFRLL, 3=FWUOA, 0=NQcTA, 6=QYcJU, 9=bBNBO, 7=JDNXE}
{2=TFRLL, 3=FWUOA, 0=NQcTA, 7=JDNXE, 6=QYcJU, 9=bBNBO}
```

**WeakHashMap DEMO**

```java
class Element {
    private String ident;
    Element(String id) { this.ident = id; }
    public int hashCode() {
        return ident.hashCode();
    }
    public String toString() {
        return ident;
    }
    public boolean equals(Object obj) {
        return obj instanceof Element && ident.equals(((Element) obj).ident);
    }
    protected void finalize() throws Throwable {
        System.out.println("Finalizing " + getClass().getSimpleName() 
                            + " " + ident);
    }
}
class Key extends Element {
    Key(String id) { super(id);}
}
class Value extends Element {
    Value(String id) { super(id);}
}
```

我们创建一个元素类 Element，其重载了 hashCode、equals和 finalize 方法。使用这个类继承了两个子类。

```java
Key[] keys = new Key[1000];
WeakHashMap<Key,Value> map = new WeakHashMap<>();
for (int i = 0; i < 1000; i++){
    Key k = new Key(Integer.toString(i));
    Value v = new Value(Integer.toString(i));
    if (i % 3 == 0){
        keys[i] = k; //添加引用，在WeakMap中进行垃圾回收时不至于清除
    }
    map.put(k,v);
}
print(map);
System.gc(); //启动垃圾回收，那些没有引用的对象全部清除了
print(map); //现在只剩下了带有引用的那些，也就是3的倍数

{901=901, 255=255, 902=902, 256=256...}
{255=255, 900=900, 906=906, 903=903...}
```

接下来创建一个WeakHashMap，在这个Map中，添加1000个Key类，然后，如果是3的倍数，那么同时把这个引用放在一个数组中。执行 gc 之后，发现，除了 3 的倍数的元素被留下来，其余的全部被清空了。

WeakHashMap自动处理那些没有引用的对象，这在对内存要求紧张的情况下十分有用。

下面探究一下它是如何工作的：

其实 WeakHashMap 内部保存的并非对象，而是 WeakReference 引用。

当一个对象，没有了引用，那么系统就会判定其需要进行垃圾回收，只要一个对象是可获得的，那么系统的垃圾回收器就不能回收它。

如果你没有持有某个对象，但是又想让垃圾管理器不着急释放它，而是紧急情况，比如内存占用过大再去释放。我们将这个对象存入 Reference 中，然后引用这个 Reference 引用。Reference 有三种类型：SoftReference、WeakReference、PhantomReference，其可获得性由强到弱。

ReferenceQueue是一个回收前用作处理的工具，当满足一定条件后，可以选择将这三个 Reference 对象放置到 ReferenceQueue 中，垃圾处理器会在任何时候处理这个 Queue 里的对象。如果被加入Queue之后，那么将不允许复活，不能使用 get 方法获取元素。

这样，如果系统负载正常，那么我们可以丢失对对象的直接持有，而是持有一个保存这个对象的 Reference的引用，并从中获得这个对象。如果条件恶劣，那么自动将 Reference 中的对象移动到 Queue 中准备进行垃圾回收，降低系统负担。这样就提供给对象一个延迟处理的机会，以备需要时重新从 Reference 中获取，或者不需要时从 Reference 转移到 Queue 中，兵直接丢弃。

```java
class VeryBig {
    private static final int SIZE = 10000;
    private long[] la = new long[SIZE];
    private String ident;
    VeryBig(String s){ this.ident = s; }
    public String toString() {
        return super.toString() + " " + ident;
    }
    protected void finalize() throws Throwable {
        System.out.println(getClass().getSimpleName() + " " + ident);
    }
}
```

上述构造了一个类，这个类的对象占用空间很大。

```java
class References {
    private static int count = 0;
    private static ReferenceQueue<VeryBig> rq = new ReferenceQueue<>();
    public static void checkQueue(){
        java.lang.ref.Reference<? extends VeryBig> inq = rq.poll();
        count++;
        if (inq != null){
            System.out.println("In quenue: "+inq.get() + " " + count);
        }

    }
    public static void main(String[] args){
        //对于Soft而言，因为其内存占用过高，所以当没有直接引用时，会将其添加到Queue中进行清除。
        LinkedList<SoftReference<VeryBig>> sa = new LinkedList<>();
        for (int i = 0; i < 1000; i++){
            sa.add(new SoftReference<VeryBig>(new VeryBig("Weak:"+i),rq));
            checkQueue(); //没有被放入Queue中，直接清除了
        }
        System.gc();
        print(rq);
                               
        //对于Weak而言，如果其保有Soft或者本身引用，则不清楚，反之，如果内存过高，则添加到Queue中
        LinkedList<WeakReference<VeryBig>> wa = new LinkedList<>();
        for (int i = 0; i < 1000; i++){
            wa.add(new WeakReference<VeryBig>(new VeryBig("Weak:"+i),rq));
            checkQueue();
        }
        System.gc(); //可以看到，被添加到Queue中了，不可从Queue中直接获取对象并提取
    }
}
```

上文介绍了 Queue 和 SoftReference 的使用，注意这三个 Ref 的区别。

- 对于SReference：只要对于对象的直接引用还在，就不会清除ref。而当没有引用且内存占用高，则随时会清除ref，将其添加到queue中
- 对于WReference：当直接对象引用不再，且引用此对象的SReference也不存在的话，则同上
- 对于PReference：当直接引用不在，没有其S/W引用时，则同上


## 实现 Hash 和 equals 方法

对于我们自己的类，一定要实现 HashCode 和 equals 方法。

```java
import java.util.*; import static com.mazhangjing.Print.*;

class Toy {
    final int num;
    Toy(int num) { this.num = num;}
    public int hashCode() {
        return num;
    }
    public boolean equals(Object obj) {
        if (obj.getClass().getSimpleName() == "Toy"){
            return ((Toy)obj).num == this.num;
        } else return super.equals(obj);
    }
}
Toy a = new Toy(3);
Toy b = new Toy(5);
Toy c = new Toy(2);
Toy e = new Toy(2);
printf("%b %b\n",c==e,c.equals(e)); // false true
//equals的实现用于比较对象

HashSet<Toy> hs = new HashSet<>();
hs.addAll(Arrays.asList(a,b,c));
print(hs); //[Toy@2, Toy@3, Toy@5]
```

**equals介绍**

一个正确的equals必须满足：

- 【逻辑性】equals(null) 永远为false
- 【对称性】a equals b == b equals a
- 【自反性】a euqals a 永远为true
- 【一致性】对于一次判断，无论进行多少次，结果都一样
- 【传递性】a equals b, b equals c, 那么 c 一定 equals a

**HashCode介绍**

在Hashxxx内部，容器使用数组保存数据。当我们查询一个 HashMap 的 Key 的时候，先去计算这个 Key 的散列值（int），然后用其作为下标到数组中寻找对应的值的 list，使用 equals 进行比较，找到这个散列值对应列表中对应的值，然后返回给我们。

因此，hashCode的生成不能依赖易变的信息，否则这个数据就无法找到，比如，一个实例是String的 hashCode 明显是基于String内容的：

```java
String[] s = {"corkine","corkine","ma"};
printf("%d, %d, %d\n",s[0].hashCode(),s[1].hashCode(),s[2].hashCode()); 
//954960411, 954960411, 3476
```

此外，hashcode应该尽量均匀分布，否则负载会很重。

HashMAP&hashSet可以通过参数传递设置桶位数和负载因子，其含义如下：

初始容量 指的是 初始桶位数，容量 指的是 桶位数，负载因子空表示 0 负载，0.5 表示半满的表，负载越轻，那么发生冲突可能性就越小，那么不用 equals 比较list，速度就很快。当负载达到规定水平时，Java自动增多桶位数（容量加倍）。默认负载因子为 0.75。如果负载因子过大，那么内存占用越多，但查找越快，如果负载过小，那么内存占用少，但是查找慢。

通常来说，Hashxxx总是最优的选择，除非你有特殊的需求，比如顺序，那么选择tree，比如插入或者迭代，那么选择linked。


## Collections 实用方法概述

Collections中含有大量的实用方法，比如快速添加、搜索、类型检查、填充、最大，最小、打乱顺序、交换、反序、线程同步。

```java
//下面展示Java容器类的fail-fast快速报错，比如一个线程在写，而另一个线程在读这种情况
List<String> list = new ArrayList<>(Arrays.asList("hello","world"));
Iterator<String> i = list.iterator();
list.add("news"); //获取完迭代器后又添加元素，当然失败，这是 fail-fast 线程安全的例子
print(i.next());

Exception in thread "main" java.util.ConcurrentModificationException
at java.base/java.util.ArrayList$Itr.checkForComodification
```